In [18]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [2]:
with open("datasets/loans.txt", "r") as f:
    loans = json.load(f)

In [3]:
loans

{'data': [[1,
   'CAB3E0EA-54FF-4502-902A-092B10ECDAF8',
   1,
   1456413152,
   '390945',
   1456413152,
   '390945',
   None,
   '2014-04-24T00:00:00',
   'Accion',
   None,
   None,
   None,
   None,
   'Chicago',
   'IL',
   '60621',
   None,
   'Other Services',
   None,
   None,
   [None, None, None, None, None]],
  [2,
   '83DC4C30-85A5-4306-8C59-8558C087C1FB',
   2,
   1456413152,
   '390945',
   1456413152,
   '390945',
   None,
   '2014-12-12T00:00:00',
   'Accion',
   '101XX',
   'S',
   'Lafayette',
   'Street',
   'Chicago',
   'IL',
   '60628',
   '9',
   'Construction/Repair',
   '41.709285897',
   '-87.624925274',
   [None, '41.7092858973584', '-87.62492527406924', None, False]],
  [3,
   'CF850AD1-880F-4BAB-8F04-0A82BB939E20',
   3,
   1456413152,
   '390945',
   1456413152,
   '390945',
   None,
   '2013-09-06T00:00:00',
   'Accion',
   '101XX',
   'S',
   'S',
   'Lafayette',
   'Chicago',
   'IL',
   '60628',
   '9',
   'Construction/Repair',
   None,
   None,
   [N

In [4]:
type(loans)

dict

it's technically a hashtable/dictionary with key value pairs. loans is a nested json object with data and meta keys as parent tags. we're just interested in the data tag....for now. let's extract the data from the data tag. 

In [5]:
for tags in loans:
    print(tags)

meta
data


The loans data needs to be serialized (flattened) into an array so that it can be read as a list. The json_normalize method in pandas does this. json_normalize returns a pandas dataframe out of the json format data file. 

In [7]:
result = pd.io.json.json_normalize(loans, "data")

In [9]:
result.head(2).transpose()

,0,1
0,1,2
1,CAB3E0EA-54FF-4502-902A-092B10ECDAF8,83DC4C30-85A5-4306-8C59-8558C087C1FB
2,1,2
3,1456413152,1456413152
4,390945,390945
5,1456413152,1456413152
6,390945,390945
7,None,None
8,2014-04-24T00:00:00,2014-12-12T00:00:00
9,Accion,Accion


Notice the last column is not fully flattened yet. The value itself is an array. 

In [14]:
def flatten_json(input_doc):
    out = {} # empty dictionary
    
    def flatten(x, name = ""):
        if type(x) is dict:
            for a in x:
                flaten(x[a], name + a + '_')
        elif type(x) is list:
                i = 0
                for a in x:
                    flatten(a, name + str(i) + "_")
                    i += 1
        else:
                    out[name[:-1]] = x

    flatten(input_doc)
    return out

In [15]:
type(loans["data"][0])

list

In [16]:
type(loans["data"][0][21])

list

In [17]:
flatten_json(loans["data"][0])

{'0': 1,
 '1': 'CAB3E0EA-54FF-4502-902A-092B10ECDAF8',
 '10': None,
 '11': None,
 '12': None,
 '13': None,
 '14': 'Chicago',
 '15': 'IL',
 '16': '60621',
 '17': None,
 '18': 'Other Services',
 '19': None,
 '2': 1,
 '20': None,
 '21_0': None,
 '21_1': None,
 '21_2': None,
 '21_3': None,
 '21_4': None,
 '3': 1456413152,
 '4': '390945',
 '5': 1456413152,
 '6': '390945',
 '7': None,
 '8': '2014-04-24T00:00:00',
 '9': 'Accion'}

In [21]:
final_data = pd.DataFrame()

for i in np.arange(0, len(loans["data"])):
    final_data = final_data.append(pd.DataFrame(json_normalize(flatten_json(loans["data"][i]))))

In [22]:
final_data.head()

,0,1,10,11,12,13,14,15,16,17,...,21_2,21_3,21_4,3,4,5,6,7,8,9
0,1,CAB3E0EA-54FF-4502-902A-092B10ECDAF8,None,None,None,None,Chicago,IL,60621,None,...,None,None,None,1456413152,390945,1456413152,390945,None,2014-04-24T00:00:00,Accion
0,2,83DC4C30-85A5-4306-8C59-8558C087C1FB,101XX,S,Lafayette,Street,Chicago,IL,60628,9,...,-87.62492527406924,None,False,1456413152,390945,1456413152,390945,None,2014-12-12T00:00:00,Accion
0,3,CF850AD1-880F-4BAB-8F04-0A82BB939E20,101XX,S,S,Lafayette,Chicago,IL,60628,9,...,None,None,None,1456413152,390945,1456413152,390945,None,2013-09-06T00:00:00,Accion
0,4,6AD8DB81-0B26-45E3-9EAD-A890524AD7E9,010XX,E,43rd,Street,Chicago,IL,60653,4,...,-87.59981435844077,None,False,1456413152,390945,1456413152,390945,None,2015-10-08T00:00:00,WBDC
0,5,B9E8523F-3973-401F-AEA6-570D87582718,010XX,E,76th,Street,Chicago,IL,60619,8,...,-87.59881954520198,None,False,1456413152,390945,1456413152,390945,None,2014-11-20T00:00:00,Accion


In [23]:
final_data[:1]

,0,1,10,11,12,13,14,15,16,17,...,21_2,21_3,21_4,3,4,5,6,7,8,9
0,1,CAB3E0EA-54FF-4502-902A-092B10ECDAF8,None,None,None,None,Chicago,IL,60621,None,...,None,None,None,1456413152,390945,1456413152,390945,None,2014-04-24T00:00:00,Accion


In [24]:
final_data.drop(final_data.columns[[0,1,12,14,15,16,17,18,19,20,21,22,23]],inplace=True,axis=1)
# df.columns[[0,1,12,14,15,16,17,18,19,20,21,22,23]]
final_data[:1]

,10,11,12,13,14,15,16,17,18,19,20,8,9
0,None,None,None,None,Chicago,IL,60621,None,Other Services,None,None,2014-04-24T00:00:00,Accion


Now let's set the column names. Luckily for us, the column names are in the meta tag. 

In [25]:
len(loans["meta"]["view"]["columns"])

22

In [26]:
names = []

In [27]:
length = len(loans["meta"]["view"]["columns"])

for i in range(0, length):
    names.append(loans["meta"]["view"]["columns"][i]["fieldName"])
    
names

[':sid',
 ':id',
 ':position',
 ':created_at',
 ':created_meta',
 ':updated_at',
 ':updated_meta',
 ':meta',
 'loan_date',
 'lender',
 'block',
 'street_direction',
 'street_name',
 'street_type',
 'city',
 'state',
 'zip',
 'ward',
 'industry',
 'latitude',
 'longitude',
 'location']

In [28]:
names = [word for word in names if not ":" in word]
names

['loan_date',
 'lender',
 'block',
 'street_direction',
 'street_name',
 'street_type',
 'city',
 'state',
 'zip',
 'ward',
 'industry',
 'latitude',
 'longitude',
 'location']

In [29]:
names.pop(len(names) - 1)

'location'

In [30]:
final_data.rename(index=str, columns={'8': names[0], '9': names[1], '10': names[2], '11': names[3], '12': names[4], '13': names[5],\
                              '14':names[6], '15': names[7], '16': names[8], '17': names[9], '18': names[10], \
                              '19':names[11], '20': names[12]},inplace=True)
final_data[:5]

,block,street_direction,street_name,street_type,city,state,zip,ward,industry,latitude,longitude,loan_date,lender
0,None,None,None,None,Chicago,IL,60621,None,Other Services,None,None,2014-04-24T00:00:00,Accion
0,101XX,S,Lafayette,Street,Chicago,IL,60628,9,Construction/Repair,41.709285897,-87.624925274,2014-12-12T00:00:00,Accion
0,101XX,S,S,Lafayette,Chicago,IL,60628,9,Construction/Repair,None,None,2013-09-06T00:00:00,Accion
0,010XX,E,43rd,Street,Chicago,IL,60653,4,Retail Sales - Other,41.817000516,-87.599814358,2015-10-08T00:00:00,WBDC
0,010XX,E,76th,Street,Chicago,IL,60619,8,Beauty/Salon Services,41.75705433,-87.598819545,2014-11-20T00:00:00,Accion
